# Regresión Lineal vs Árbol de decisión


In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
from pathlib import Path
# Esta es otra forma de crear directorios 
# IMAGES_PATH es un objeto que permite hacer llamadas al sistema
# como por ejemplo, crear un directorio
IMAGES_PATH = Path() / "images" / "regression_trees"
# parents: crear directorios superiores o no
# exist_ok: qué pasa si ya existe
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor


In [ ]:
import pandas as pd
penguins = pd.read_csv("data/penguins_pre.csv")

In [ ]:
X = penguins.drop(["body_mass_g"], axis=1)
X.rename(columns={"target":"species"}, inplace=True)
y = penguins["body_mass_g"]


In [ ]:
# Hacer una función con todo y probar diversos arboles StratifiedShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, X["species"]):
    train_set = X.iloc[train_index]
    train_y_set = y.iloc[train_index]
    test_set = X.iloc[test_index]
    test_y_set = y.iloc[test_index]

In [ ]:
# Entrenar regresión linal
lr = LinearRegression()


In [ ]:
# Interpretar coeficientes

In [ ]:
# Predecir el primer elemento del test y comprobar que es correcto

In [ ]:
# Entrenar modelo
tree_reg = DecisionTreeRegressor(max_depth=4, random_state=42)


In [ ]:
from sklearn.tree import export_graphviz
from graphviz import Source
export_graphviz(
    tree_reg,
    out_file=str(IMAGES_PATH / "regression_tree.dot"),
    feature_names=train_set.columns,
    rounded=True,
    filled=True
)
Source.from_file(IMAGES_PATH / "regression_tree.dot")

In [ ]:
# Interpretar gráfico

In [ ]:
# Hacer una predicción del primer elemento del test y comprobar

In [ ]:
# Comparar el error train y test
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
# ¿Qué pasa si cambiamos la semilla/orden de los datos en cada modelo?

In [ ]:
# Comprueba el árbol de decisión sin ningún hiperparámetro